In [1]:
# from mobile_net import get_model
import librosa
import tensorflow as tf
import glob
import numpy as np
import random
import wave
import os
import pickle
import noisereduce as nr

2024-08-15 07:43:56.402271: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-15 07:43:58.067308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 07:43:58.658499: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 07:43:58.819623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 07:44:00.037878: I tensorflow/core/platform/cpu_feature_guar

In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_logical_devices('GPU')

2024-08-15 07:44:11.253571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43629 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:8b:00.0, compute capability: 8.9


[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
noise_path = '/shareddrive/working/data_code/data/neg_data/_background_noise_/chunks/1s_chunks/*'
environment_path = '/shareddrive/working/data_code/data/neg_data/envornment/chunks/1s_chunks/*'
word_path = '/shareddrive/working/data_code/data/neg_data/spcmd/all_words/original/*'
recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/1s_chunks/*'

In [4]:
word_files = glob.glob(word_path)
environment_files = glob.glob(environment_path)
noise_files = glob.glob(noise_path)
recording_files = glob.glob(recording_path)
len(environment_files),len(noise_files),len(word_files),len(recording_files)

(1109, 660, 64721, 5806)

In [5]:
def get_duration(audio_path):
    with wave.open(audio_path, 'rb') as wav_file:
        sample_rate = wav_file.getframerate()
        num_frames = wav_file.getnframes()
        duration = num_frames / sample_rate
    return duration

In [6]:
limit = 1.5
adele_path = '/shareddrive/working/data_code/data/adele/augmented/*'
adele_folders = glob.glob(adele_path)
adele_files = [j for i in adele_folders for j in glob.glob(f'{i}/*')]
# adele_files = [j for i in adele_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]
hilfe_path = '/shareddrive/working/data_code/data/hilfe/augmented/*'
hilfe_folders = glob.glob(hilfe_path)
hilfe_files = [j for i in hilfe_folders for j in glob.glob(f'{i}/*')]
# hilfe_files = [j for i in hilfe_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]

In [7]:
len(adele_files),len(hilfe_files)

(10950, 10935)

In [8]:
no_of_files = 1109+660
file_path_and_labels = list()
# file_path_and_labels.extend([(i,1) for i in random.sample(adele_files,no_of_files)])
file_path_and_labels.extend([(i,1) for i in adele_files])
# file_path_and_labels.extend([(i,2) for i in random.sample(hilfe_files,no_of_files)])
# file_path_and_labels.extend([(i,2) for i in hilfe_files])
# avg_files = (len(adele_files) + len(hilfe_files)) // 2
avg_files = len(adele_files)
# file_path_and_labels.extend([(i,0) for i in random.sample(recording_files,len(recording_files)//2)])
file_path_and_labels.extend([(i,0) for i in random.sample(word_files,avg_files)])
file_path_and_labels.extend([(i,0) for i in noise_files])
file_path_and_labels.extend([(i,0) for i in environment_files])
random.shuffle(file_path_and_labels)
avg_files

10950

In [9]:
avg_files-len(recording_files)//2

8047

In [10]:
sr = 16000
max_seconds = 1
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))

def process_data(y,sr,max_seconds):
    y = pad_or_trunc(y,sr*max_seconds)
    features = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=1024)
    return features
    
def get_processed_data(audio_file):
    y,_ = librosa.load(audio_file,sr=sr)
    # y = nr.reduce_noise(y=y, sr=sr,n_fft=1024)
    # y[np.isnan(y)] = 0
    features = process_data(y,sr,max_seconds)
    return features
try:
    with open('f_and_l.pickle','rb') as f:
        features_and_labels = pickle.load(f)
except:
    features_and_labels = list()
    for i,j in file_path_and_labels:
        try:
            features = get_processed_data(i)
            features_and_labels.append((features,j))
        except: print(i)
        # features = get_processed_data(i)
        # features_and_labels.append(features,j)
    # with open('f_and_l.pickle','wb') as f:
    #     pickle.dump(features_and_labels,f)
finally:
    print(features_and_labels[0][0].shape)

(128, 32)


In [11]:


class DataSequenceRaw(tf.keras.utils.Sequence):

    def __init__(self,data,batch_size):
        self.data_size = len(data)
        audios,labels = zip(*data)
        self.X = np.array(audios)
        self.Y = np.array(labels)
        self.batch_size = batch_size

        
    def __len__(self):
        return int(np.ceil(self.data_size / self.batch_size))

    def __getitem__(self,idx):
        s = idx * self.batch_size
        e = (idx + 1) * self.batch_size
        X = self.X[s:e]
        Y = self.Y[s:e]
        return X,Y


In [12]:
def _get_train_val_size(total_examples,train_percent=70):
    
    train_ratio = round(train_percent)/100
    remaining_percent = 100 - train_percent
    test_percent = round(remaining_percent/3)*2
    val_percent = remaining_percent - test_percent
    val_ratio = val_percent/100
    train_size = round(total_examples * train_ratio)
    val_size = round(total_examples * val_ratio)
    return train_size,val_size

def get_data_raw(data,train_percent=70,batch_size=32):

    train_size,val_size = _get_train_val_size(len(data),train_percent)

    train_examples = data[:train_size]
    val_examples = data[train_size:train_size+val_size]
    test_examples = data[train_size+val_size:]

    train = DataSequenceRaw(train_examples,batch_size=batch_size)
    test = DataSequenceRaw(test_examples,batch_size=1)
    val = DataSequenceRaw(val_examples,batch_size=batch_size)

    return train,test,val

train,test,val = get_data_raw(features_and_labels,train_percent=80)

In [13]:
shape = train[0][0][0].shape
input_shape = [*shape,1]

In [15]:
def get_model(
        input_shape,
        output_neurons=1,
        output_activation='sigmoid',
        loss=tf.keras.losses.binary_crossentropy,
        lr=0.0001
):
    _input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv1D(200,kernel_size=3,padding='valid',activation='relu')(_input)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.GRU(150,return_sequences=False)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512,activation='relu')(x)
    x = tf.keras.layers.Dense(64,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10,activation='relu')(x)
    outputs = tf.keras.layers.Dense(output_neurons,activation=output_activation,kernel_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
    model = tf.keras.Model(inputs=_input,outputs=outputs)

    model.compile(
        loss=loss,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'],
    )

    return model

model = get_model(
        input_shape=shape,
        lr=0.001
)



In [16]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=5,mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=1,mode='max',restore_best_weights=True,start_from_epoch=10)
with tf.device('/gpu'):
    history = model.fit(train,epochs=100,validation_data=val,verbose=1,callbacks=[reduce_lr,early_stopping])

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-15 08:07:11.715897: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1723709232.110750     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709232.157786     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709232.158025     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709232.158237     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709232.179911  

 16/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5394 - loss: 0.7184

W0000 00:00:1723709234.001542     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.001866     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.004498     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.005884     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.008679     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.018067     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.018293     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.020704     188 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709234.020918     188 gp

528/592 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5364 - loss: 0.7090

W0000 00:00:1723709237.462528     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.462775     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.462954     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.463141     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.463319     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.463516     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.463707     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.463894     190 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709237.464079     190 gp

592/592 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.5363 - loss: 0.7083 - val_accuracy: 0.5451 - val_loss: 0.6918 - learning_rate: 0.0010
Epoch 2/100
  1/592 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.5000 - loss: 0.6988

W0000 00:00:1723709238.310132     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.328714     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.328911     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.329088     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.329262     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.329478     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.329662     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.329850     186 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1723709238.330041     186 gp

592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5292 - loss: 0.6899 - val_accuracy: 0.5451 - val_loss: 0.6906 - learning_rate: 0.0010
Epoch 3/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5336 - loss: 0.6876 - val_accuracy: 0.5451 - val_loss: 0.6499 - learning_rate: 0.0010
Epoch 4/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.5362 - loss: 0.6835 - val_accuracy: 0.5451 - val_loss: 0.5570 - learning_rate: 0.0010
Epoch 5/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7452 - loss: 0.5272 - val_accuracy: 0.9500 - val_loss: 0.2792 - learning_rate: 0.0010
Epoch 6/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9413 - loss: 0.2940 - val_accuracy: 0.9613 - val_loss: 0.1925 - learning_rate: 0.0010
Epoch 7/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9580 - loss: 0.2070 - val_accuracy: 0.9796 - val_loss: 0.1348 - learning_rate: 0.0010
Epoch 8/100
592/592 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9716 - loss: 0.1546 - val_a

In [19]:
model.evaluate(test)

3314/3314 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9821 - loss: 0.0669


[0.07048802822828293, 0.9815932512283325]

In [20]:
folder_path = '/shareddrive/working/model_code/models/custom_model_2/trail_1'
os.makedirs(folder_path,exist_ok=True)
model_path = f'{folder_path}/16k_melspec-nfft-1024_a_cnn_gru_dense_model.keras'
model.save(model_path)